# Downloading data from Google Drive

In [1]:
#!pip install gdown


In [2]:
#import gdown
# to download the data from Google Drive
# id = 1XKcjj7eEAmgmbHU4Qa7g2fUbHHXRkCBi   ( saber_11_pro.csv)
#url = 'https://drive.google.com/uc?export=download&id=1XKcjj7eEAmgmbHU4Qa7g2fUbHHXRkCBi'
#output = 'saber_11_pro.csv'
#gdown.download(url, output, quiet=False)


In [3]:
#import gdown
# to download the data from Google Drive
# id = 10OhmdA_gI3EDYj0IS0uUKsYDot_IBUnu    (saber_combined_all_fields.csv)
#url ='https://drive.google.com/uc?export=download&id=10OhmdA_gI3EDYj0IS0uUKsYDot_IBUnu'
#output = 'saber_combined_all_fields.csv'
#gdown.download(url, output, quiet=False)

In [4]:
#import gdown
# to download the data from Google Drive
# id = 1HOBcz5K_QfeYavdDl2iCkJGAhlh1SSJV    (saber_combined_all_fields.csv)
#url ='https://drive.google.com/uc?export=download&id=1HOBcz5K_QfeYavdDl2iCkJGAhlh1SSJV'
#output = 'saber_combined_preprocessed.csv'
#gdown.download(url, output, quiet=False)

# Libraries

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

import plotly.express as px
import plotly.graph_objs as go

from sklearn import preprocessing

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score,  r2_score,  f1_score, classification_report,roc_curve,roc_auc_score


 
from sklearn import model_selection  # Kfold..
# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

import warnings

pd.set_option('display.max_rows', 500) # to show more rows.
warnings.simplefilter('ignore')



# DF, Dataset : Saber_Combined_Preprocessed

In [6]:
df = pd.read_csv('../input/saber-combined-preprocessed/saber_combined_preprocessed.csv')
df.head()

In [7]:
def nan_check(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
    return missing_data

#nan_check(df)

In [8]:
#df.describe().T.style.bar(subset=['mean'], color='#FF595E')\
#                           .background_gradient(subset=['50%'], cmap='PiYG') # highlight median

In [9]:
# Categorical features
cat_col=df.select_dtypes(include='object').columns.to_list()
#cat_col, len(cat_col)


In [10]:
# Numerical features
num_col=df.select_dtypes(include='number').columns.to_list()
#num_col, len(num_col)

In [11]:
%%time
### Corralation matrix heatmap
# Getting correlation matrix
cor_matrix = df[num_col].corr().round(1)
mask = np.triu(np.ones_like(cor_matrix, dtype = bool))
# Plotting heatmap 
fig = plt.figure(figsize=(25,25));
plt.title('Correlation matrix for df Saber Combined PreProcessed')
sns.heatmap(cor_matrix, mask=mask, annot=True, center=0,linewidths = .5,square=True,cbar_kws={"shrink": .60}, cmap = sns.diverging_palette(250, 10, as_cmap=True), ax=plt.subplot(111));
plt.show()

# Building the Recommendation System

In [12]:
recsys_df = df.copy()
recsys_df.head()

In [13]:
# Categorical features
cat_col=recsys_df.select_dtypes(include='object').columns.to_list()
#cat_col, len(cat_col)


In [14]:
# Numerical features
num_col=recsys_df.select_dtypes(include='number').columns.to_list()
#num_col, len(num_col)

In [15]:
features = [
   # numeric features of scores
    'score_math_saber_pro',
    'score_language_saber_pro',
    'score_social_science_saber_pro',
    'score_english_saber_pro',
    'score_writing_saber_pro',
    ## 
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
]

In [16]:
recsys_df[features].head()

In [17]:
recsys_df[features].describe().T.style.bar(subset=['mean'], color='#FF595E')\
                           .background_gradient(subset=['50%'], cmap='PiYG') # highlight median

In [18]:
%%time
### Corralation matrix heatmap
# Getting correlation matrix
cor_matrix = recsys_df[features].corr().round(1)
mask = np.triu(np.ones_like(cor_matrix, dtype = bool))
# Plotting heatmap 
fig = plt.figure(figsize=(15,15));
plt.title('Correlation matrix for df Saber Combined PreProcessed')
sns.heatmap(cor_matrix, mask=mask, annot=True, center=0,linewidths = .5,square=True,cbar_kws={"shrink": .60}, cmap = sns.diverging_palette(250, 10, as_cmap=True), ax=plt.subplot(111));
plt.show()

These features are all Correlated 0.9
* 'score_math_saber_pro',
*  'score_language_saber_pro',
*  'score_social_science_saber_pro',
*  'score_english_saber_pro',
* 'score_writing_saber_pro',

In [19]:
features = [
    ## 
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
]

In [20]:
%%time
### Corralation matrix heatmap
# Getting correlation matrix
cor_matrix = recsys_df[features].corr().round(1)
mask = np.triu(np.ones_like(cor_matrix, dtype = bool))
# Plotting heatmap 
fig = plt.figure(figsize=(15,15));
plt.title('Correlation matrix for df Saber Combined PreProcessed')
sns.heatmap(cor_matrix, mask=mask, annot=True, center=0,linewidths = .5,square=True,cbar_kws={"shrink": .60}, cmap = sns.diverging_palette(250, 10, as_cmap=True), ax=plt.subplot(111));
plt.show()

Check distribution of all numerical variables 

In [21]:
for var in features:
    print(var.upper() + ' Distribution')
    print(recsys_df[var].describe())
    print('-----')

Note that the max-min range of all 3 variables are very different from one another. When we do further analysis, like multivariate linear regression, for example, the attributed income will intrinsically influence the result more due to its larger value. Hence, it is important to standardize and normalize the data to bring all variables to the same range.

I use a Robust Scaler (QuantileTransformer): similar to normalization but it instead uses the interquartile range, so that it is robust to outliers.

In [22]:
from sklearn.preprocessing import QuantileTransformer

qt = QuantileTransformer(output_distribution='normal')

In [23]:
recsys_df['score_language_saber_11'] = qt.fit_transform(recsys_df[['score_language_saber_11']].values.reshape(-1,1))
recsys_df['score_mathematics_saber_11'] = qt.fit_transform(recsys_df[['score_mathematics_saber_11']].values.reshape(-1,1))
recsys_df['score_biology_saber_11'] = qt.fit_transform(recsys_df[['score_biology_saber_11']].values.reshape(-1,1))
recsys_df['score_chemistry_saber_11'] = qt.fit_transform(recsys_df[['score_chemistry_saber_11']].values.reshape(-1,1))
recsys_df['score_physics_saber_11'] = qt.fit_transform(recsys_df[['score_physics_saber_11']].values.reshape(-1,1))
recsys_df['score_social_science_saber_11'] = qt.fit_transform(recsys_df[['score_social_science_saber_11']].values.reshape(-1,1))
recsys_df['score_philosophy_saber_11'] = qt.fit_transform(recsys_df[['score_philosophy_saber_11']].values.reshape(-1,1))
recsys_df['score_english_saber_11'] = qt.fit_transform(recsys_df[['score_english_saber_11']].values.reshape(-1,1))

**distribution after transformation**

In [24]:
for var in features:
    print(var.upper() + ' Distribution')
    print(recsys_df[var].describe())
    print('-----')

**Check distribution of scaled variables**

In [25]:
for cat in features:
    cat_num = recsys_df[cat]
    print("Plot for %s: total counts = %d" % (cat.upper(), len(cat_num)))
    f, ax = plt.subplots(figsize=(10,5))
    plot = sns.distplot(cat_num)
    plot.set_xticklabels(plot.get_xticklabels(), rotation=45)
    plt.show()

**Distribution of the Target Variable = Undergraduate Core**

In [26]:
f, ax = plt.subplots(figsize=(20,5))
plot = sns.countplot(recsys_df['your_undergraduate_core'],)
plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
plt.show()

**As we can see there are many Cores that should be eliminated from the data**

In [27]:
#create a mapping from labels to a unique integer and vice versa for labelling and prediction later
labels = recsys_df['your_undergraduate_core'].unique()
i = 0
idx2class = {} 
class2idx = {}
for tp in labels:
    idx2class[i] = tp
    class2idx[tp] = i
    i += 1
idx2class

In [28]:
class2idx

In [29]:
# Final features
features_train = [
      'score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',
      # target
     'your_undergraduate_core'
]


In [30]:
recsys = recsys_df[features_train]
recsys.head()

In [31]:
recsys['label'] = recsys['your_undergraduate_core'].replace(class2idx)
recsys.head()

When a dataset is imbalanced, i.e., the distribution of target variables is not uniform, we should make sure that the training-validation split is stratified. 

This ensures that the distribution of the target variable is preserved in both the training and validation datasets.

In [32]:
def stratified_split(df, target, val_percent=0.2):
    '''
    Function to split a dataframe into train and validation sets, while preserving the ratio of the labels in the target variable
    Inputs:
    - df, the dataframe
    - target, the target variable
    - val_percent, the percentage of validation samples, default 0.2
    Outputs:
    - train_idxs, the indices of the training dataset
    - val_idxs, the indices of the validation dataset
    '''
    classes=list(df[target].unique())
    train_idxs, val_idxs = [], []
    for c in classes:
        idx=list(df[df[target]==c].index)
        np.random.shuffle(idx)
        val_size=int(len(idx)*val_percent)
        val_idxs+=idx[:val_size]
        train_idxs+=idx[val_size:]
    return train_idxs, val_idxs

In [33]:
train_idxs, val_idxs = stratified_split(recsys, 'label', val_percent=0.25)

val_idxs, test_idxs = stratified_split(recsys[recsys.index.isin(val_idxs)], 'label', val_percent=0.5)

In [34]:
def test_stratified(df, col):
    '''
    Analyzes the ratio of different classes in a categorical variable within a dataframe
    Inputs:
    - dataframe
    - categorical column to be analyzed
    Returns: None
    '''
    classes=list(df[col].unique())
    
    for c in classes:
        print(f'Proportion of records with {c}: {len(df[df[col]==c])*1./len(df):0.2} ({len(df[df[col]==c])} / {len(df)})')

In [35]:
print('---------- STRATIFIED SAMPLING REPORT ----------')
print('-------- Label proportions in FULL data --------')
test_stratified(recsys, 'label')
print('-------- Label proportions in TRAIN data --------')
test_stratified(recsys[recsys.index.isin(train_idxs)], 'label')
print('------ Label proportions in VALIDATION data -----')
test_stratified(recsys[recsys.index.isin(val_idxs)], 'label')
print('-------- Label proportions in TEST data ---------')
test_stratified(recsys[recsys.index.isin(test_idxs)], 'label')

In [36]:
train_df = recsys[recsys.index.isin(train_idxs)]
X_train = train_df[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_train = train_df[['label']].values
print('Retrieved Training Data')
val_df = recsys[recsys.index.isin(val_idxs)]
X_val = val_df[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_val = val_df[['label']].values
print('Retrieved Validation Data')
test_df = recsys[recsys.index.isin(test_idxs)]
X_test = test_df[['score_language_saber_11',
      'score_mathematics_saber_11',
      'score_biology_saber_11',
      'score_chemistry_saber_11',
      'score_physics_saber_11',
      'score_social_science_saber_11',
      'score_philosophy_saber_11',
      'score_english_saber_11',]].values
Y_test = test_df[['label']].values
print('Retrieved Test Data')

# Neural Network

In [37]:
# multi-class classification with Keras
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#from tensorflow.keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y_train = encoder.transform(Y_train)
encoded_Y_val = encoder.transform(Y_val)
encoded_Y_test = encoder.transform(Y_test)


# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = tf.keras.utils.to_categorical(encoded_Y_train)
dummy_y_val = tf.keras.utils.to_categorical(encoded_Y_val)
dummy_y_test = tf.keras.utils.to_categorical(encoded_Y_test)

In [38]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=8, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(64, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','MeanSquaredError','AUC','top_k_categorical_accuracy'])
    return model


In [39]:
model = baseline_model()
model.summary()

In [40]:
%%time
history = model.fit(X_train,dummy_y_train, validation_data=(X_val,dummy_y_val),epochs= 20,batch_size=100,verbose=1)

In [41]:
print('Baseline Neural Network using Tensorflow Keras, with 3 layers(2 Relu, Softmax) (input = 8, output = 64), 20 epochs, Adam optimizer, \n')
print('------------\n')
print('Model Accuracy = {}'.format(history.history['accuracy'][np.argmin(history.history['loss'])]*100))
print('Model TOP 5 Accuracy = {}'.format(history.history['top_k_categorical_accuracy'][np.argmin(history.history['loss'])]*100))
print('Model AUC = {}'.format(history.history['auc'][np.argmin(history.history['loss'])]*100))
print('Model RMSE = {}'.format(history.history['mean_squared_error'][np.argmin(history.history['loss'])]))
print('Model loss-categorical_crossentropy  = {}'.format(history.history['loss'][np.argmin(history.history['loss'])]))

In [42]:
plt.figure(figsize=(8,6))
plt.title('Accuracy scores')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])

plt.figure(figsize=(8,6))
plt.title('AUC value')
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.legend(['auc', 'val_auc'])

plt.figure(figsize=(8,6))
plt.title('top_k_categorical_accuracy value')
plt.plot(history.history['top_k_categorical_accuracy'])
plt.plot(history.history['val_top_k_categorical_accuracy'])
plt.legend(['top_k_categorical_accuracy', 'val_top_k_categorical_accuracy'])

plt.figure(figsize=(8,6))
plt.title('RMSE value')
plt.plot(history.history['mean_squared_error'])
plt.plot(history.history['val_mean_squared_error'])
plt.legend(['mean_squared_error', 'val_mean_squared_error'])

plt.figure(figsize=(8,6))
plt.title('Loss value')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [43]:
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(Y_test, y_pred_bool))

**Labels are imbalanced..**
# Balance data with imbalance learn module

In [44]:
# import library
#import imblearn
#from imblearn.under_sampling import RandomUnderSampler
#from imblearn.over_sampling import RandomOverSampler
#from collections import Counter

# Random Over Sampling ( Upsize the classes)
#ros = RandomOverSampler(random_state=42)

# Random Under Sampling (Downsize the class)
#rus = RandomUnderSampler(random_state=42, replacement=True)


# OverSampling fit predictor and target varaible
#x_ros_train, y_ros_train = ros.fit_resample(X_train, Y_train)
#x_ros_val,   y_ros_val = ros.fit_resample(X_val, Y_val)
#x_ros_test,  y_ros_test = ros.fit_resample(X_test, Y_test)

# UnderSampling fit predictor and target varaible
#x_rus_train, y_rus_train = rus.fit_resample(X_train, Y_train)
#x_rus_val, y_rus_val = rus.fit_resample(X_val, Y_val)
#x_rus_test, y_rus_test = rus.fit_resample(X_test, Y_test)
#print('Resample Train dataset shape', Counter(y_ros_train))
#print('------')
#print('Resample Val dataset shape', Counter(y_ros_val))
#print('------')
#print('Resample Test dataset shape', Counter(y_ros_test))
# With OverSampling we are about 1.7 Million...
#len(x_ros_train)

# Models Building

## 1. Logistic Regression
## 2. Random Forest
## 3. XGBoost

For simplicity, I will explore Logistic regression(OneVsRest) as the baseline model and progressively use more complicated ensemble models for our prediction task such as XGBoost.

In [45]:
from sklearn.model_selection import GridSearchCV
#store data, all in numpy arrays
training_data = {'X_train':X_train,'Y_train':Y_train,
                'X_val': X_val,'Y_val':Y_val,
                'X_test': X_test,'Y_test':Y_test}

## 1. Logistic Regression

In [50]:
%%time
from sklearn.linear_model import LogisticRegression
# define the multinomial logistic regression model
lr_multi = LogisticRegression(multi_class='multinomial', solver='sag') # error with solver lbfgs , reference : https://stackoverflow.com/questions/65682019/attributeerror-str-object-has-no-attribute-decode-in-fitting-logistic-regre


In [51]:
%%time
lr_multi.fit(training_data['X_train'], training_data['Y_train'].reshape(training_data['Y_train'].shape[0]))

In [52]:
predicted_labels_lr = lr_multi.predict(training_data['X_test'])

In [ ]:
#predicted_lr = lr_multi.predict(training_data['X_test'])
#fpr_lr, tpr_lr, thresholds_lr = roc_curve(training_data['Y_test'], predicted_lr)  ## ValueError: multiclass format is not supported
#auc_lr = auc(fpr_lr, tpr_lr)

In [53]:
print('LR Multi Accuracy = {}'.format(accuracy_score(training_data['Y_test'], predicted_labels_lr)*100))
print('LR Multi F1-Score (Micro): {}'.format(f1_score(training_data['Y_test'], predicted_labels_lr,average='micro')*100))
#print('RF AUC = {}'.format(auc_rf)) # commented till fix error above
print('LR Multi RMSE = {}'.format(mean_squared_error(training_data['Y_test'], predicted_labels_lr,squared=False)))

In [57]:
f, ax = plt.subplots(figsize=(10,5))
plot = sns.barplot(x=features, y=lr_multi.coef_[0])
ax.set_title('LR Multi Feature Importance')
plot.set_xticklabels(plot.get_xticklabels(),rotation='vertical')
plt.show()


## 2. Random Forest¶

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_jobs=-1,random_state=10, verbose=1,bootstrap=False,class_weight='balanced')
rf_clf.fit(training_data['X_train'], training_data['Y_train'].reshape(training_data['Y_train'].shape[0]))

In [ ]:
predicted_labels_rf = rf_clf.predict(training_data['X_test'])

In [ ]:
#predicted_ls = rf_clf.predict(training_data['X_val']).ravel()
#fpr_rf, tpr_rf, thresholds_ = roc_curve(training_data['Y_test'], predicted_ls)
#auc_rf = auc(fpr_rf, tpr_rf)

In [ ]:
print('RF Accuracy = {}'.format(accuracy_score(training_data['Y_test'], predicted_labels_rf)*100))
print('RF F1-Score (Micro): {}'.format(f1_score(training_data['Y_test'], predicted_labels_rf,average='micro')*100))
#print('RF AUC = {}'.format(auc_rf))
print('RF RMSE = {}'.format(mean_squared_error(training_data['Y_test'], predicted_labels_rf,squared=False)))

In [ ]:
print(classification_report(training_data['Y_val'], predicted_labels_rf))

In [ ]:
f, ax = plt.subplots(figsize=(10,5))
plot = sns.barplot(x=features, y=rf_clf.feature_importances_)
ax.set_title('RF Feature Importance')
plot.set_xticklabels(plot.get_xticklabels(),rotation='vertical')
plt.show()


## 3. XGBoost

In [ ]:
import xgboost as xgb
# allow logloss and classification error plots for each iteraetion of xgb model
def plot_compare(metrics,eval_results,epochs):
    for m in metrics:
        test_score = eval_results['val'][m]
        train_score = eval_results['train'][m]
        rang = range(0, epochs)
        plt.rcParams["figure.figsize"] = [6,6]
        plt.plot(rang, test_score,"c", label="Val")
        plt.plot(rang, train_score,"orange", label="Train")
        title_name = m + " plot"
        plt.title(title_name)
        plt.xlabel('Iterations')
        plt.ylabel(m)
        lgd = plt.legend()
        plt.show()
        
def fitXgb(sk_model, training_data=training_data,epochs=300):
    print('Fitting model...')
    sk_model.fit(training_data['X_train'], training_data['Y_train'].reshape(training_data['Y_train'].shape[0],))
    print('Fitting done!')
    train = xgb.DMatrix(training_data['X_train'], label=training_data['Y_train'])
    val = xgb.DMatrix(training_data['X_val'], label=training_data['Y_val'])
    params = sk_model.get_xgb_params()
    metrics = ['mlogloss','merror']
    params['eval_metric'] = metrics
    store = {}
    evallist = [(val, 'val'),(train,'train')]
    xgb_model = xgb.train(params, train, epochs, evallist,evals_result=store,verbose_eval=100)
    fpr_xgb, tpr_xgb, thresholds_xgb = roc_curve(y_test, sk_model.predict(training_data['X_test']))
    print('-- Model Report --')
    print('XGBoost Accuracy: {}'.format(accuracy_score(sk_model.predict(training_data['X_test']), training_data['Y_test'])))
    print('XGBoost F1-Score (Micro): {}'.format(f1_score(sk_model.predict(training_data['X_test']),training_data['Y_test'],average='micro')))
    print('XGBoost AUC: {}'.format(auc(fpr_xgb, tpr_xgb)))
    print('XGBoost RMSE: {}'.format(mean_squared_error(sk_model.predict(training_data['X_test']), training_data['Y_test'])))
    plot_compare(metrics,store,epochs)
    features = ['score_language_saber_11', 'score_mathematics_saber_11',
      'score_biology_saber_11',  'score_chemistry_saber_11',
      'score_physics_saber_11',  'score_social_science_saber_11',
      'score_philosophy_saber_11',  'score_english_saber_11']
    f, ax = plt.subplots(figsize=(10,5))
    plot = sns.barplot(x=features, y=sk_model.feature_importances_)
    ax.set_title('Feature Importance')
    plot.set_xticklabels(plot.get_xticklabels(),rotation='vertical')
    plt.show()

### XGBoost hyperparameter tuning with GridSearchCV
#### Initial model

In [ ]:
from xgboost.sklearn import XGBClassifier
#initial model
xgb1 = XGBClassifier(learning_rate=0.1,
                    n_estimators=400,
                    max_depth=5,
                    min_child_weight=1,
                    gamma=0,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    #objective='multi:softmax',
                    nthread=4,
                    tree_method='gpu_hist',
                    num_class=64,
                    seed=27)

In [ ]:
%%time
fitXgb(xgb1, training_data)